Run the requirements.txt file to download dependencies needed

# Run this cell if you want to use Ollama engine:

In [ ]:
# RUN THIS CELL IF YOU WANT TO USE OLLAMA ENGINE
from llama_index.llms.ollama import Ollama
from llama_index.core.indices.struct_store import JSONQueryEngine

# llm = Ollama(model="llama3", request_timeout=120.0)

# Test setup
# resp = llm.complete("What is Mcdonalds?")
# print(resp)

# Begin vector embedding of scrapped data

In [ ]:
import json

# Function to read JSON data from a file
def read_json_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

In [ ]:
# Paths to the JSON files
list_of_json_files = [
    "cp_data/tenant_desc.json",
    "cp_data/tenant_schema.json",
    "cp_data/about_us.json",
    "cp_data/news.json",
    "cp_data/facilities_desc.json",
    "cp_data/whats_hot.json",
]

In [ ]:
from llama_index.readers.json import JSONReader

reader = JSONReader()

# Load data from multiple files
documents = []
for file_path in list_of_json_files:
    documents.extend(reader.load_data(file_path))

for doc in documents:
    print("Document ID:", doc.doc_id)

In [ ]:
# Create an index over the documents
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

# Setup MilvusVectorStore
vector_store = MilvusVectorStore(
    dim=1536,
    collection_name="cp_tenant_vector",
    uri="./milvus_demo.db",
    token="root:Milvus",
    overwrite=True
)

# Create a storage context with the Milvus vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index from your documents
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

# Uncomment this cell to test project setup 

In [ ]:
# Uncomment to Run Test:

# query_engine = index.as_query_engine(llm=llm) #uncomment to use ollama llm
# query_engine = index.as_query_engine()

# res1 = query_engine.query("What is CHATERAISE category and where is its location")
# res2 = query_engine.query("Who is the US landscape consultant for central park mall")
# print(res1)
# print(res2)

# Run this cell for FAST API deployment and running RAG with RESTFUL env:

This call only if you choose to run Fast API with ngrok or you can deploy locally

!ngrok authtoken "YOUR API KEY HERE"

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from threading import Thread
from pyngrok import ngrok
import uvicorn
import logging

class QueryResponse(BaseModel):
    response: str

class QueryRequest(BaseModel):
    query: str

# Initialize the FastAPI app
app = FastAPI()

llama_index = query_engine

@app.post("/query", response_model=QueryResponse)
async def query_llama_index(request: QueryRequest):
    try:
        # Logging the received request
        logging.info(f"Received query: {request.query}")

        # Extract the 'query' value from the request body
        query = request.query
        result = llama_index.query(query)
        if isinstance(result, str):
            return QueryResponse(response=result)
        else:
            return QueryResponse(response=result.response)
    except Exception as e:
        logging.error(f"Error occurred: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {str(e)}")


# Function to run Uvicorn server
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the server in a background thread
server = Thread(target=run)
server.start()

# Expose the server to the public internet via ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")


# Kill the server to FAST API deployment

In [ ]:
ngrok.disconnect(public_url=public_url)
ngrok.kill()